# Day 21: Keypad Conundrum

In [ ]:
import numpy as np
from itertools import permutations

def read_data(is_test: bool = False):
    if is_test:
        with open('data/2024-21-example.txt', 'r') as f:
            data = f.readlines()
    else:
        with open('data/2024-21.txt', 'r') as f:
            data = f.readlines()
    
    data = [x.strip() for x in data]
    return data

In [48]:
# Positions and directions as numpy vectors
POSITIONS = {
    # real numpad positions
    "7": np.array([0, 0]),
    "8": np.array([0, 1]),
    "9": np.array([0, 2]),
    "4": np.array([1, 0]),
    "5": np.array([1, 1]),
    "6": np.array([1, 2]),
    "1": np.array([2, 0]),
    "2": np.array([2, 1]),
    "3": np.array([2, 2]),
    "0": np.array([3, 1]),
    "A": np.array([3, 2]),
    # robot numpad positions
    "^": np.array([0, 1]),
    "a": np.array([0, 2]),
    "<": np.array([1, 0]),
    "v": np.array([1, 1]),
    ">": np.array([1, 2]),
}

DIRECTIONS = {
    "^": np.array([-1, 0]),
    "v": np.array([1, 0]),
    "<": np.array([0, -1]),
    ">": np.array([0, 1]),
}

In [54]:
memoization = {}

def get_movement_perms(start_pos: np.ndarray, end_pos: np.ndarray, avoid: np.ndarray = np.array([0, 0])):
    # String of needed moves to go from start_pos to end_pos
    movement = end_pos - start_pos
    mov_str = ""
    if movement[0] < 0:
        mov_str += "^" * abs(movement[0])
    else:
        mov_str += "v" * movement[0]
    if movement[1] < 0:
        mov_str += "<" * abs(movement[1])
    else:
        mov_str += ">" * movement[1]

    # Generate unique permutations of moves,
    # ignoring ones that would include the avoid position
    perms = [
        "".join(p) + "a"
        for p in set(permutations(mov_str))
        if not any(
            (sum(DIRECTIONS[move] for move in p[:i]) +  start_pos == avoid).all()
            for i in range(len(p))
        )
    ]
    return perms if perms else ["a"]


def get_min_length(s, lim=2, depth=0):
    # save length of certain moves to avoid recalculating
    key = (s, depth, lim)
    if key in memoization:
        return memoization[key]

    # define avoid position and starting position
    avoid = np.array([3, 0]) if depth == 0 else np.array([0, 0])
    cur = POSITIONS["A"] if depth == 0 else POSITIONS["a"]
    length = 0

    for char in s:
        nextCurrent = POSITIONS[char]
        moveSet = get_movement_perms(cur, nextCurrent, avoid)
        if depth == lim:
            length += len(moveSet[0])
        else:
            length += min(get_min_length(move, lim, depth + 1) for move in moveSet)
        cur = nextCurrent

    memoization[key] = length
    return length


def compute_solution(data, limit: int):
    total_cost = 0
    for code in data:
        code_val = int(code[:3])
        min_length = get_min_length(code, limit)
        price = code_val * min_length
        total_cost += price
        print(f"{code_val} * {min_length} = {price}")
    return total_cost

In [ ]:
data = read_data(is_test=False)
print('## Part 1 ##')
total_cost = compute_solution(data, 2)
print('Part 1:', total_cost)

print('\n## Part 2 ##')
total_cost = compute_solution(data, 25)
print('Part 2:', total_cost)